In [1]:
from conf import databases, url, data_dir
from file_utils import create_folder
from nn import LasagneNN
from preproc import shuffled_examples

import numpy
import random
import time

import lasagne
from matplotlib import pyplot as plt
import matplotlib as mpl

Using gpu device 0: GeForce GTX 1050 Ti (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)
/home/jdu/anac3.5/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
databases = [databases[0]]
print(databases)

[('mitdb', ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234'])]


In [3]:
fs = 128
fs_target = 360
min_bpm = 10
max_bpm = 350
min_gap = fs*60/min_bpm
max_gap = fs*60/max_bpm

train_perc = 99
test_perc = 100 - train_perc

params = {
    'fs_target': 360,
    'segment_size': 5000, 
    'segment_step': 2500, 
    'normalized_steps': True,
    'correct_peaks': True,
    'left_border': 50, # samples
    'right_border': 50, # samples
    'min_gap': min_gap,
    'max_gap': max_gap
}

In [4]:
model_path = 'saved-models/170518'
create_folder(model_path)
save_name = model_path + '/model_1'

In [5]:
from lasagne.nonlinearities import rectify
architecture = [
    (0.5, [
            (3, 8, rectify, 'same'),
            (15, 64, rectify, 'same'),
            (45, 64, rectify, 'same'),
            (89, 32, rectify, 'same'),
            (149, 16, rectify, 'same'),
            (199, 16, rectify, 'same'),
            (299, 16, rectify, 'same'),
    ]),
    (0.5, [
            (3, 8, rectify, 'same'),
            (9, 64, rectify, 'same'),
            (19, 64, rectify, 'same'),
            (39, 64, rectify, 'same'),
    ]),
    (0.5, [
            (3, 8, rectify, 'same'),
            (5, 32, rectify, 'same'),
            (9, 32, rectify, 'same'),
            (15, 32, rectify, 'same'),
            (19, 32, rectify, 'same'),
    ]),
    (0.5, [
            (3, 8, rectify, 'same'),
            (5, 32, rectify, 'same'),
            (15, 64, rectify, 'same'),
    ]),
    (0.5, [
            (5, 16, rectify, 'same'),
            (15, 16, rectify, 'same'),
            (19, 16, rectify, 'same'),
    ]),
    (0.5, [
            (3, 8, rectify, 'same'),
            (9, 16, rectify, 'same'),
            (15, 16, rectify, 'same'),
    ])
]

nn = LasagneNN(architecture, dim=1, params=params)

Model shape:
  Layer 0:
    - size: 3	num: 8
    - size: 15	num: 64
    - size: 45	num: 64
    - size: 89	num: 32
    - size: 149	num: 16
    - size: 199	num: 16
    - size: 299	num: 16
    - dropout: 0.5
  Layer 1:
    - size: 3	num: 8
    - size: 9	num: 64
    - size: 19	num: 64
    - size: 39	num: 64
    - dropout: 0.5
  Layer 2:
    - size: 3	num: 8
    - size: 5	num: 32
    - size: 9	num: 32
    - size: 15	num: 32
    - size: 19	num: 32
    - dropout: 0.5
  Layer 3:
    - size: 3	num: 8
    - size: 5	num: 32
    - size: 15	num: 64
    - dropout: 0.5
  Layer 4:
    - size: 5	num: 16
    - size: 15	num: 16
    - size: 19	num: 16
    - dropout: 0.5
  Layer 5:
    - size: 3	num: 8
    - size: 9	num: 16
    - size: 15	num: 16
    - dropout: 0.5
Neural network initialized in 24.12s


In [6]:
exs = shuffled_examples(data_dir, databases, 360, 5000, 2500, True, True)
train_exs = exs[:int(len(exs)*train_perc/100)]
test_exs  = exs[int(len(exs)*train_perc/100):]
print('Number of examples: {} ({} samples)'.format(len(exs), len(exs)*5000))
print('          Training: {} ({} samples)'.format(len(train_exs), len(train_exs)*5000))
print('           Testing: {} ({} samples)'.format(len(test_exs), len(test_exs)*5000))

NameError: name 'numpy' is not defined

In [ ]:
nn.train(train_exs, test_exs, num_epochs=500, examples_by_epoch=1000, save_name=save_name, eval_during_training=False)

In [ ]:
_, params = load_model(model_path + '/model-loss0.037631392673427445-epoch1200.sav')
# print(params)
set_all_param_values(nn.model, params)
nn.init_funs(nn.model)

In [ ]:
eval_model(testXY, nn.evaluate, left_border=left_border, right_border=right_border,
           min_gap=min_gap, max_gap=max_gap, plot_examples=True, nb=10,
           nearest_fpr=0.000675, threshold=0.9678147, eval_margin=10)